In [1]:
python_code = '''
import json
import sys
import random

json.dump(
    [('heads' if random.randint(0,1) == 1 else 'tails') for i in range(10)],
    open('/tmp/output.json', 'w')
)        
'''    

In [2]:
import kfp
from kfp import dsl 

@dsl.pipeline(name='Loop pipeline',description='A pipeline with parallel loop')
def loop_pipeline():
    flips = flip_coins_op()
    with dsl.ParallelFor(flips.output) as item:
        print_op(item)   

def flip_coins_op():
    return dsl.ContainerOp(
        name='Flip coin',
        image='python:alpine3.6',
        command=['sh', '-c'],
        arguments=[f'python -c "{python_code}"'],
        file_outputs={'output': '/tmp/output.json'}
    )

def print_op(msg):
    return dsl.ContainerOp(
        name='Print',
        image='alpine:3.6',
        command=['echo', msg],
    )

In [3]:
pipelineGzFile = 'hahahoho.zip'

kfp.compiler.Compiler().compile(
    loop_pipeline, 
    pipelineGzFile
)

client = kfp.Client()
my_exp = client.create_experiment(
    name='Basic Experiment'
)
my_run = client.run_pipeline(
    my_exp.id, 
    'Loop pipeline',
    pipelineGzFile
)